<a href="https://colab.research.google.com/github/Rayzon3/Natural-Language-Processing/blob/main/Sentiment_140_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install ktrain

In [77]:
import numpy as np
import pandas as pd
import re

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import transformers
import ktrain
from ktrain import text

In [85]:
import gdown

# Replace the Google Drive link with your own link
google_drive_url = 'https://drive.google.com/uc?id=0B04GJPshIjmPRnZManQwWEdTZjg'
# https://drive.google.com/file/d/0B04GJPshIjmPRnZManQwWEdTZjg/view?usp=drive_link&resourcekey=0-betyQkEmWZgp8z0DFxWsHw

# Define the output file path
output_file = 'your_file'

# Download the file
gdown.download(google_drive_url, output_file, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=0B04GJPshIjmPRnZManQwWEdTZjg
To: /content/your_file
100%|██████████| 81.4M/81.4M [00:00<00:00, 251MB/s]


'your_file'

In [88]:
!ls

drive  sample_data  sentiment140.csv  your_file  your_file.csv


In [89]:
!unzip your_file

Archive:  your_file
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [102]:
data = pd.read_csv('/content/testdata.manual.2009.06.14.csv', sep=',', header=None, encoding='latin')

In [103]:
data.head()

,0,1,2,3,4,5
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...


In [104]:
labeled_data = data[[0, 5]]

In [105]:
labeled_data[0] = labeled_data[0].apply(lambda x: 0 if x == 0 else 1)

<ipython-input-105-8e29f95c37a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_data[0] = labeled_data[0].apply(lambda x: 0 if x == 0 else 1)


In [106]:
labeled_data.columns = ['label', 'text']

In [107]:
labeled_data.head()

,label,text
0,1,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,1,Reading my kindle2... Love it... Lee childs i...
2,1,"Ok, first assesment of the #kindle2 ...it fuck..."
3,1,@kenburbary You'll love your Kindle2. I've had...
4,1,@mikefish Fair enough. But i have the Kindle2...


In [108]:
hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls = re.compile(r"https?://\S+")

def process_text(text):
  text = hashtags.sub(' hashtag', text)
  text = mentions.sub(' entity', text)
  return text.strip().lower()

def match_expr(pattern, string):
  return not pattern.search(string) == None

def remove_urls(dataset):
    link_with_urls = dataset.text.apply(lambda x: match_expr(urls, x))
    return dataset[[not e for e in link_with_urls]]

link_with_urls = labeled_data.text.apply(lambda x: match_expr(urls, x))

In [110]:

from sklearn.model_selection import train_test_split
TRAIN_SIZE = 0.20
VAL_SIZE = 0.05
dataset_count = len(labeled_data)

df_train_val, df_test = train_test_split(labeled_data, test_size=1-TRAIN_SIZE-VAL_SIZE, random_state=42)
df_train, df_val = train_test_split(df_train_val, test_size=VAL_SIZE / (VAL_SIZE + TRAIN_SIZE), random_state=42)

print("TRAIN size:", len(df_train))
print("VAL size:", len(df_val))
print("TEST size:", len(df_test))

TRAIN size: 99
VAL size: 25
TEST size: 374


In [111]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df=df_train,
                   text_column='text',
                   label_columns='label',
                   val_df=df_val,
                   maxlen=400,
                   preprocess_mode='bert',)

['not_label', 'label']
     not_label  label
166        0.0    1.0
156        0.0    1.0
212        1.0    0.0
439        0.0    1.0
99         1.0    0.0
['not_label', 'label']
     not_label  label
279        0.0    1.0
53         0.0    1.0
217        1.0    0.0
363        0.0    1.0
427        0.0    1.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [112]:
X_train[0].shape

(99, 400)

In [113]:
#Making Model
model = text.text_classifier(name='bert',
                             train_data=(X_train, y_train),
                             preproc=preprocess)

Is Multi-Label? False
maxlen is 400


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


done.


In [114]:
#Learning Rate
learner = ktrain.get_learner(model=model,
                             train_data=(X_train, y_train),
                             val_data=(X_test, y_test),
                             batch_size=6)

In [115]:
#Training
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
17/17 [==============================] - 44s 877ms/step - loss: 0.7882 - accuracy: 0.4949 - val_loss: 0.5416 - val_accuracy: 0.7600


In [116]:
learner

In [117]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [125]:
#Test
data = ['Everything is going good!']

In [126]:
sentiment = predictor.predict(data)
if sentiment[0] is 'label':
  print('Positive')
else:
  print('Negative')

Positive


<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-126-6e07588b6287>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sentiment[0] is 'label':


In [120]:
data = ["Today's day really sucked"]

In [121]:
sentiment = predictor.predict(data)
if sentiment[0] is 'label':
  print('Positive')
else:
  print('Negative')

<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?


Positive


<ipython-input-121-6e07588b6287>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sentiment[0] is 'label':


In [122]:
predictor.save('bert')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
